## 127 Introduction to DML and Patitioning

In [3]:
val username = System.getProperty("user.name")

username = itversity


itversity

In [4]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder.enableHiveSupport.appName("127 DML and Partitioning").getOrCreate

spark = org.apache.spark.sql.SparkSession@cc40928


org.apache.spark.sql.SparkSession@cc40928

In [6]:
%%sql

USE itversity_retail

++
||
++
++



In [7]:
%%sql

SHOW TABLES

+----------------+-----------+-----------+
|        database|  tableName|isTemporary|
+----------------+-----------+-----------+
|itversity_retail|order_items|      false|
|itversity_retail|     orders|      false|
|itversity_retail|   students|      false|
+----------------+-----------+-----------+



In [8]:
%%sql

DROP TABLE orders

++
||
++
++



In [11]:
%%sql

CREATE TABLE orders(
    order_id INT,
    order_date STRING,
    order_customer_id INT,
    order_status STRING
)ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

++
||
++
++



In [13]:
%%sql

SELECT current_database()

+------------------+
|current_database()|
+------------------+
|  itversity_retail|
+------------------+



In [14]:
%%SQL

LOAD DATA LOCAL INPATH '/data/retail_db/orders' INTO TABLE orders

++
||
++
++



In [15]:
%%sql

select count(1) from orders

+--------+
|count(1)|
+--------+
|   68883|
+--------+



## 128 Introduction to Partitioning

In [17]:
%%sql

SHOW TABLES

+----------------+-----------+-----------+
|        database|  tableName|isTemporary|
+----------------+-----------+-----------+
|itversity_retail|order_items|      false|
|itversity_retail|     orders|      false|
|itversity_retail|   students|      false|
+----------------+-----------+-----------+



In [18]:
 %%sql

DROP TABLE if exists order_items 

++
||
++
++



In [19]:
%%sql

CREATE TABLE order_items(
    order_item_id INT,
    order_item_order_id INT,
    order_item_product_id INT,
    order_item_quantity INT,
    order_item_subtotal FLOAT,
    order_item_product_price FLOAT
)stored as parquet

++
||
++
++



In [20]:
spark.sql("DESCRIBE FORMATTED order_items").show(200,false)

+----------------------------+------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                     |comment|
+----------------------------+------------------------------------------------------------------------------+-------+
|order_item_id               |int                                                                           |null   |
|order_item_order_id         |int                                                                           |null   |
|order_item_product_id       |int                                                                           |null   |
|order_item_quantity         |int                                                                           |null   |
|order_item_subtotal         |float                                                                         |null   |
|order_item_product_price    |float                     

In [23]:
import sys.process._

s"hdfs dfs -ls /user/${username}/warehouse/${username}_retail.db/order_items"!

0

## Difference between Load and Insert

In [27]:
%%sql

load data LOCAL INPATH '/data/retail_db/order_items/' INTO TABLE order_items

++
||
++
++



In [29]:
%%sql
SELECT * FROM order_items limit 10

	at org.apache.parquet.hadoop.ParquetFileReader.re...


Magic sql failed to execute with error: 
Job aborted due to stage failure: Task 0 in stage 5.0 failed 4 times, most recent failure: Lost task 0.3 in stage 5.0 (TID 18, itvdelab, executor 2): java.lang.RuntimeException: hdfs://localhost:9000/user/itversity/warehouse/itversity_retail.db/order_items/part-00000 is not a Parquet file. expected magic number at tail [80, 65, 82, 49] but found [48, 46, 48, 10]
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:524)
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:505)
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:499)
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:448)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anonfun$buildReaderWithPartitionValues$1.footerFileMetaData$lzycompute$1(ParquetFileFormat.scala:371)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileForm

## 131 Inserting Data using stage table leveraging

In [30]:
%%sql

truncate table order_items

++
||
++
++



In [31]:
%%sql
SELECT COUNT(1) FROM order_items

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [35]:
%%sql

CREATE TABLE order_items_stage(
    order_item_id INT,
    order_item_order_id INT,
    order_item_product_id INT,
    order_item_quantity INT,
    order_item_subtotal FLOAT,
    order_item_product_price FLOAT
)ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

++
||
++
++



In [40]:
spark.sql("DESCRIBE FORMATTED order_items_stage").show(200,false)

+----------------------------+------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                           |comment|
+----------------------------+------------------------------------------------------------------------------------+-------+
|order_item_id               |int                                                                                 |null   |
|order_item_order_id         |int                                                                                 |null   |
|order_item_product_id       |int                                                                                 |null   |
|order_item_quantity         |int                                                                                 |null   |
|order_item_subtotal         |float                                                                               |null   |
|order_i

In [42]:
%%sql

truncate table order_items_stage

++
||
++
++



In [43]:
%%sql

load data LOCAL INPATH '/data/retail_db/order_items/' INTO TABLE order_items_stage

++
||
++
++



In [44]:
%%sql 

SELECT * FROM order_items_stage limit 5

|        87945|              35212|                 1014|                  1|              49.98|    ...


+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|        87943|              35210|                 1014|                  4|             199.92|                   49.98|
|        87944|              35211|                  365|                  4|             239.96|                   59.99|
|        87945|              35212|                 1014|                  1|              49.98|                   49.98|
|        87946|              35212|                  191|                  3|             299.97|                   99.99|
|        87947|              35212|                 1014|                  5|              249.9|                   49.98|
+-------------+-

In [45]:
%%sql

truncate table order_items

++
||
++
++



In [50]:
%%sql

insert into order_items SELECT * FROM order_items_stage

++
||
++
++



In [48]:
%%sql 

SELECT * FROM order_items limit 5

|            3|                  2|                  502|                  5|              250.0|    ...


+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|             299.98|                  299.98|
|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|            3|                  2|                  502|                  5|              250.0|                    50.0|
|            4|                  2|                  403|                  1|             129.99|                  129.99|
|            5|                  4|                  897|                  2|              49.98|                   24.99|
+-------------+-

In [49]:
%%sql

SELECT COUNT(1) from order_items

+--------+
|count(1)|
+--------+
|  172198|
+--------+



In [51]:
%%sql

SELECT COUNT(1) from order_items

+--------+
|count(1)|
+--------+
|  344396|
+--------+



In [54]:
%%sql

INSERT OVERWRITE TABLE order_items SELECT * FROM order_items_stage

++
||
++
++



In [55]:
%%sql

SELECT COUNT(1) from order_items

+--------+
|count(1)|
+--------+
|  172198|
+--------+



In [56]:
%%sql 

SELECT * FROM order_items limit 5

|        87945|              35212|                 1014|                  1|              49.98|    ...


+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|        87943|              35210|                 1014|                  4|             199.92|                   49.98|
|        87944|              35211|                  365|                  4|             239.96|                   59.99|
|        87945|              35212|                 1014|                  1|              49.98|                   49.98|
|        87946|              35212|                  191|                  3|             299.97|                   99.99|
|        87947|              35212|                 1014|                  5|              249.9|                   49.98|
+-------------+-

In [57]:
import sys.process._

In [59]:
s"hdfs dfs -ls /user/${username}/warehouse/${username}_retail.db/order_items/"!

Found 3 items
-rw-r--r--   1 itversity supergroup          0 2024-02-02 10:19 /user/itversity/warehouse/itversity_retail.db/order_items/_SUCCESS
-rw-r--r--   1 itversity supergroup     862839 2024-02-02 10:19 /user/itversity/warehouse/itversity_retail.db/order_items/part-00000-dd2d518c-1104-4531-9683-291bfce225a7-c000.snappy.parquet
-rw-r--r--   1 itversity supergroup     858034 2024-02-02 10:19 /user/itversity/warehouse/itversity_retail.db/order_items/part-00001-dd2d518c-1104-4531-9683-291bfce225a7-c000.snappy.parquet


0

## 132 Creating Partitioned Table

In [62]:
%%sql

DROP TABLE IF EXISTS orders_part

++
||
++
++



In [63]:
%%sql

CREATE TABLE orders_part(
    order_id INT,
    order_date STRING,
    order_customer_id INT,
    order_status STRING
)PARTITIONED BY (ORDER_MONTH INT)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

++
||
++
++



In [64]:
%%SQL
DESCRIBE orders_part

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|            order_id|      int|   null|
|          order_date|   string|   null|
|   order_customer_id|      int|   null|
|        order_status|   string|   null|
|         ORDER_MONTH|      int|   null|
|# Partition Infor...|         |       |
|          # col_name|data_type|comment|
|         ORDER_MONTH|      int|   null|
+--------------------+---------+-------+



In [67]:
spark.sql("DESCRIBE FORMATTED orders_part").show(200,false)

+----------------------------+------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                     |comment|
+----------------------------+------------------------------------------------------------------------------+-------+
|order_id                    |int                                                                           |null   |
|order_date                  |string                                                                        |null   |
|order_customer_id           |int                                                                           |null   |
|order_status                |string                                                                        |null   |
|ORDER_MONTH                 |int                                                                           |null   |
|# Partition Information     |                          

In [68]:
import sys.process._

s"hdfs dfs -ls /user/${username}/warehouse/${username}_retail.db/orders_part"!

0

## 133 Adding partitions to Table

In [69]:
%%sql

USE itversity_retail

++
||
++
++



In [70]:
%%sql

DROP TABLE orders_part

++
||
++
++



In [71]:
%%sql

CREATE TABLE orders_part(
    order_id INT,
    order_date STRING,
    order_customer_id INT,
    order_status STRING
)PARTITIONED BY (order_month STRING)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

++
||
++
++



In [72]:
import sys.process._

s"hdfs dfs -ls -R /user/${username}/warehouse/${username}_retail.db/orders_part"!

0

In [73]:
%%sql

ALTER TABLE orders_part ADD PARTITION (order_month = '2013-07')

++
||
++
++



In [74]:
import sys.process._

s"hdfs dfs -ls -R /user/${username}/warehouse/${username}_retail.db/orders_part"!

drwxr-xr-x   - itversity supergroup          0 2024-02-02 10:34 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=2013-07


0

In [90]:
%%sql

USE itversity_retail

++
||
++
++



In [108]:
%%sql

DROP TABLE orders_part

++
||
++
++



In [109]:
%%sql

CREATE TABLE orders_part(
    order_id INT,
    order_date STRING,
    order_customer_id INT,
    order_status STRING
)PARTITIONED BY (order_month INT)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

++
||
++
++



In [110]:
%%sql

DESCRIBE orders_part

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|            order_id|      int|   null|
|          order_date|   string|   null|
|   order_customer_id|      int|   null|
|        order_status|   string|   null|
|         order_month|      int|   null|
|# Partition Infor...|         |       |
|          # col_name|data_type|comment|
|         order_month|      int|   null|
+--------------------+---------+-------+



In [111]:
import sys.process._

s"hdfs dfs -ls -R /user/${username}/warehouse/${username}_retail.db/orders_part"!

0

In [112]:
%%sql

ALTER TABLE orders_part ADD PARTITION (order_month=201307)

++
||
++
++



In [113]:
%%sql

ALTER TABLE orders_part ADD
    PARTITION (order_month=201308)
    PARTITION (order_month=201309)
    PARTITION (order_month=201310)

++
||
++
++



In [114]:
import sys.process._

s"hdfs dfs -ls -R /user/${username}/warehouse/${username}_retail.db/orders_part"!

drwxr-xr-x   - itversity supergroup          0 2024-02-02 10:47 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201307
drwxr-xr-x   - itversity supergroup          0 2024-02-02 10:47 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201308
drwxr-xr-x   - itversity supergroup          0 2024-02-02 10:47 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201309
drwxr-xr-x   - itversity supergroup          0 2024-02-02 10:47 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201310


0

## 134 Loading data into Partitioned Table

In [115]:
%%sql

use itversity_retail

++
||
++
++



In [116]:
%%sql

LOAD DATA LOCAL INPATH '/home/itversity/orders/orders_201307'
    INTO TABLE orders_part PARTITION(order_month=201307)

++
||
++
++



In [117]:
import sys.process._

s"hdfs dfs -ls -R /user/${username}/warehouse/${username}_retail.db/orders_part"!

drwxr-xr-x   - itversity supergroup          0 2024-02-02 12:08 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201307
-rwxr-xr-x   1 itversity supergroup      64737 2024-02-02 12:08 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201307/orders_201307
drwxr-xr-x   - itversity supergroup          0 2024-02-02 10:47 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201308
drwxr-xr-x   - itversity supergroup          0 2024-02-02 10:47 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201309
drwxr-xr-x   - itversity supergroup          0 2024-02-02 10:47 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201310


0

In [118]:
%%sql

LOAD DATA LOCAL INPATH '/home/itversity/orders/orders_201308'
    INTO TABLE orders_part PARTITION(order_month=201308)

++
||
++
++



In [119]:
%%sql

LOAD DATA LOCAL INPATH '/home/itversity/orders/orders_201309'
    INTO TABLE orders_part PARTITION(order_month=201309)

++
||
++
++



In [122]:
%%sql

LOAD DATA LOCAL INPATH '/home/itversity/orders/orders_201310'
    overwrite INTO TABLE orders_part PARTITION(order_month=201310)

++
||
++
++



In [123]:
import sys.process._

s"hdfs dfs -ls -R /user/${username}/warehouse/${username}_retail.db/orders_part"!

drwxr-xr-x   - itversity supergroup          0 2024-02-02 12:08 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201307
-rwxr-xr-x   1 itversity supergroup      64737 2024-02-02 12:08 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201307/orders_201307
drwxr-xr-x   - itversity supergroup          0 2024-02-02 12:09 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201308
-rwxr-xr-x   1 itversity supergroup     243190 2024-02-02 12:09 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201308/orders_201308
drwxr-xr-x   - itversity supergroup          0 2024-02-02 12:09 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201309
-rwxr-xr-x   1 itversity supergroup     251262 2024-02-02 12:09 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201309/orders_201309
drwxr-xr-x   - itversity supergroup          0 2024-02-02 12:10 /user/itversity/warehouse/itversity_retail.db/or

0

In [124]:
import sys.process._

s"hdfs dfs -tail /user/${username}/warehouse/${username}_retail.db/orders_part/order_month=201310/orders_201310"!

ETE
67755,2013-10-30 00:00:00.0,8386,PENDING_PAYMENT
67756,2013-10-31 00:00:00.0,6146,PROCESSING
67757,2013-10-31 00:00:00.0,441,CLOSED
67758,2013-10-31 00:00:00.0,6750,CLOSED
67759,2013-10-31 00:00:00.0,10759,PENDING_PAYMENT
68725,2013-10-01 00:00:00.0,7795,PROCESSING
68726,2013-10-02 00:00:00.0,8817,PENDING_PAYMENT
68727,2013-10-05 00:00:00.0,5880,PENDING_PAYMENT
68728,2013-10-07 00:00:00.0,7267,COMPLETE
68729,2013-10-09 00:00:00.0,8043,PENDING_PAYMENT
68730,2013-10-11 00:00:00.0,3568,PENDING_PAYMENT
68731,2013-10-13 00:00:00.0,8102,PENDING_PAYMENT
68732,2013-10-14 00:00:00.0,9990,COMPLETE
68733,2013-10-16 00:00:00.0,12429,CLOSED
68734,2013-10-18 00:00:00.0,8510,ON_HOLD
68735,2013-10-21 00:00:00.0,788,COMPLETE
68736,2013-10-23 00:00:00.0,8462,COMPLETE
68737,2013-10-26 00:00:00.0,10302,PROCESSING
68738,2013-10-27 00:00:00.0,1100,COMPLETE
68739,2013-10-28 00:00:00.0,2528,PENDING
68740,2013-10-29 00:00:00.0,10691,ON_HOLD
68741,2013-10-30 00:00:00.0,5974,PENDING_PAYMENT
68742,2013-10-31 

0

In [127]:
%%sql

SELECT * FROM orders_part LIMIT 5

+--------+--------------------+-----------------+---------------+-----------+
|order_id|          order_date|order_customer_id|   order_status|order_month|
+--------+--------------------+-----------------+---------------+-----------+
|     778|2013-07-29 00:00:...|             5759|       COMPLETE|     201307|
|     779|2013-07-29 00:00:...|             2004|PENDING_PAYMENT|     201307|
|     780|2013-07-29 00:00:...|             2016|       COMPLETE|     201307|
|     781|2013-07-29 00:00:...|             7971|        PENDING|     201307|
|     782|2013-07-29 00:00:...|            12213|       COMPLETE|     201307|
+--------+--------------------+-----------------+---------------+-----------+



## 135 Inserting data into Partitions Table 

In [128]:
%%sql

USE itversity_retail

++
||
++
++



In [129]:
%%sql

ALTER TABLE orders_part ADD PARTITION (order_month=201311)

++
||
++
++



In [130]:
%%sql

SELECT COUNT(1) FROM orders

+--------+
|count(1)|
+--------+
|   68883|
+--------+



In [132]:
%%sql

SELECT COUNT(1) FROM orders_part

+--------+
|count(1)|
+--------+
|   18389|
+--------+



In [134]:
%%sql

INSERT INTO orders_part PARTITION(order_month=201311) SELECT * FROM orders WHERE order_date LIKE '2013-11%'

++
||
++
++



In [135]:
%%sql

SELECT COUNT(1) FROM orders_part

+--------+
|count(1)|
+--------+
|   24770|
+--------+



In [136]:
import sys.process._

s"hdfs dfs -ls -R /user/${username}/warehouse/${username}_retail.db/orders_part"!

drwxr-xr-x   - itversity supergroup          0 2024-02-02 12:08 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201307
-rwxr-xr-x   1 itversity supergroup      64737 2024-02-02 12:08 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201307/orders_201307
drwxr-xr-x   - itversity supergroup          0 2024-02-02 12:09 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201308
-rwxr-xr-x   1 itversity supergroup     243190 2024-02-02 12:09 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201308/orders_201308
drwxr-xr-x   - itversity supergroup          0 2024-02-02 12:09 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201309
-rwxr-xr-x   1 itversity supergroup     251262 2024-02-02 12:09 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201309/orders_201309
drwxr-xr-x   - itversity supergroup          0 2024-02-02 12:10 /user/itversity/warehouse/itversity_retail.db/or

0

## 136 Using Dynamic Partition while Inserting into table

In [137]:
%%sql

USE itversity_retail

++
||
++
++



In [138]:
%%sql

SHOW TABLES

+----------------+-----------------+-----------+
|        database|        tableName|isTemporary|
+----------------+-----------------+-----------+
|itversity_retail|      order_items|      false|
|itversity_retail|order_items_stage|      false|
|itversity_retail|           orders|      false|
|itversity_retail|      orders_part|      false|
|itversity_retail|         students|      false|
+----------------+-----------------+-----------+



In [139]:
%%sql

SELECT COUNT(1) FROM orders WHERE order_date like '2013-12%'

+--------+
|count(1)|
+--------+
|    5892|
+--------+



In [142]:
%%sql

SELECT COUNT(1) FROM orders

+--------+
|count(1)|
+--------+
|   68883|
+--------+



In [143]:
%%sql

SELECT COUNT(1) FROM orders_part

+--------+
|count(1)|
+--------+
|   24770|
+--------+



In [140]:
%%sql

SET hive.exec.dynamic.partition

+--------------------+-----------+
|                 key|      value|
+--------------------+-----------+
|hive.exec.dynamic...|<undefined>|
+--------------------+-----------+



In [141]:
%%sql

SET hive.exec.dynamic.partition.mode

+--------------------+-----------+
|                 key|      value|
+--------------------+-----------+
|hive.exec.dynamic...|<undefined>|
+--------------------+-----------+



In [144]:
%%sql

SET hive.exec.dynamic.partition=true

+--------------------+-----+
|                 key|value|
+--------------------+-----+
|hive.exec.dynamic...| true|
+--------------------+-----+



In [145]:
%%sql

SET hive.exec.dynamic.partition.mode=nonstrict

+--------------------+---------+
|                 key|    value|
+--------------------+---------+
|hive.exec.dynamic...|nonstrict|
+--------------------+---------+



In [147]:
%%sql

INSERT INTO orders_part PARTITION(order_month)
SELECT *, DATE_FORMAT(order_date, 'YYYYMM') order_month
FROM orders
WHERE order_date >= '2013-12-01 00:00:00:0'

++
||
++
++



In [150]:
import sys.process._

s"hdfs dfs -ls -R /user/${username}/warehouse/${username}_retail.db/orders_part"!

drwxr-xr-x   - itversity supergroup          0 2024-02-02 12:08 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201307
-rwxr-xr-x   1 itversity supergroup      64737 2024-02-02 12:08 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201307/orders_201307
drwxr-xr-x   - itversity supergroup          0 2024-02-02 12:09 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201308
-rwxr-xr-x   1 itversity supergroup     243190 2024-02-02 12:09 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201308/orders_201308
drwxr-xr-x   - itversity supergroup          0 2024-02-02 12:09 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201309
-rwxr-xr-x   1 itversity supergroup     251262 2024-02-02 12:09 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201309/orders_201309
drwxr-xr-x   - itversity supergroup          0 2024-02-02 12:10 /user/itversity/warehouse/itversity_retail.db/or

0

In [159]:
%%sql
SELECT COUNT(1) FROM orders

+--------+
|count(1)|
+--------+
|   68883|
+--------+



In [151]:
%%sql
SELECT COUNT(1) FROM orders_part

+--------+
|count(1)|
+--------+
|   68644|
+--------+



In [154]:
%%sql
SELECT DATE_FORMAT(order_date,'YYYYMM') order_month, COUNT(1) FROM orders GROUP BY DATE_FORMAT(order_date,'YYYYMM')

+-----------+--------+
|order_month|count(1)|
+-----------+--------+
|     201405|    5467|
|     201401|    5908|
|     201312|    5335|
|     201310|    5335|
|     201412|     557|
|     201311|    6381|
|     201307|    1533|
|     201407|    4468|
|     201403|    5778|
|     201404|    5657|
+-----------+--------+
only showing top 10 rows



In [156]:
%%sql
SELECT order_month, count(1) from orders_part GROUP BY order_month

+-----------+--------+
|order_month|count(1)|
+-----------+--------+
|     201402|    5635|
|     201309|    5841|
|     201406|    5308|
|     201312|    5096|
|     201403|    5778|
|     201311|    6381|
|     201405|    5467|
|     201404|    5657|
|     201308|    5680|
|     201407|    4468|
+-----------+--------+
only showing top 10 rows



In [157]:
%%sql
SELECT DATE_FORMAT(order_date,'YYYY') order_year, COUNT(1) FROM orders GROUP BY DATE_FORMAT(order_date,'YYYY')

+----------+--------+
|order_year|count(1)|
+----------+--------+
|      2014|   38778|
|      2013|   30105|
+----------+--------+



In [158]:
%%sql
SELECT DATE_FORMAT(order_date,'YYYY') order_year, COUNT(1) FROM orders_part GROUP BY DATE_FORMAT(order_date,'YYYY')

+----------+--------+
|order_year|count(1)|
+----------+--------+
|      2014|   38778|
|      2013|   29866|
+----------+--------+

